In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import warnings

# UserWarning 무시 (필요 시 제거 가능)
warnings.filterwarnings("ignore", category=UserWarning)

# 디바이스 설정 (GPU 사용 여부 확인)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Transformer 모듈 정의 (8개의 TransformerEncoderLayer 스택)
class TimesNet(nn.Module):
    def __init__(self, input_dim, seq_length, embed_dim=128, num_heads=4, num_layers=8, dropout=0.1):
        super(TimesNet, self).__init__()
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        self.input_linear = nn.Linear(input_dim, embed_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            dropout=dropout,
            batch_first=True
        )
        
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.output_linear = nn.Linear(embed_dim, embed_dim)
        self.activation = nn.ReLU()
        self.layer_norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.input_linear(x)
        x = self.transformer_encoder(x)
        x = self.output_linear(x)
        x = self.activation(x)
        x = self.layer_norm(x[:, -1, :])

        return x

# 다중 종목 주식 트레이딩 환경 정의
class MultiStockTradingEnv(gym.Env):
    def __init__(self, dfs, stock_dim=3, initial_balance=5000000, max_stock=500, seq_length=20):
        super(MultiStockTradingEnv, self).__init__()
        self.dfs = dfs  
        self.stock_dim = stock_dim  
        self.initial_balance = initial_balance  
        self.max_stock = max_stock 
        self.transaction_fee = 0.0025  
        self.slippage = 0.0005  
        self.max_loss = 0.2  
        self.seq_length = seq_length  

        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.stock_dim * (7 * self.seq_length + 1) + 1,), dtype=np.float32
        )
        self.action_space = spaces.MultiDiscrete([13] * self.stock_dim) 

        self.reset()

    def reset(self):
        self.balance = self.initial_balance
        self.portfolio_value = self.initial_balance
        self.current_step = 0
        self.stock_owned = {ticker: {'quantity': 0, 'avg_price': 0} for ticker in self.dfs.keys()}
        self.stock_price = {}
        self.total_asset = []

        self.balance_history = [self.balance]
        self.portfolio_value_history = [self.portfolio_value]
        self.action_history = []
        self.price_history = {ticker: [] for ticker in self.dfs.keys()}
        self.trade_history = []  

        self.max_steps = min(len(df) for df in self.dfs.values()) - self.seq_length - 1
        if self.max_steps <= 0:
            raise ValueError("데이터프레임의 길이가 시퀀스 길이보다 짧습니다.")

        self.data_indices = {ticker: self.seq_length for ticker in self.dfs.keys()}  

        return self._next_observation()

    def _next_observation(self):
        obs = []
        for ticker, df in self.dfs.items():
            idx = self.data_indices[ticker]
            seq = df.loc[idx - self.seq_length:idx - 1, ['Close', 'MA10', 'MA20', 'RSI', 'Volume', 'Upper_Band', 'Lower_Band']].values
            obs.extend(seq.flatten())
            obs.append(self.stock_owned[ticker]['quantity'])

            self.stock_price[ticker] = df.iloc[idx]['Close_unscaled']

        obs.append(self.balance)

        return np.array(obs, dtype=np.float32)

    def step(self, actions):
        done = False
        total_reward = 0
        invalid_action_penalty = -10  
        self.action_history.append(actions)
        trade_info = []  

        prev_portfolio_value = self.portfolio_value  

        action_mapping = {
            0: ('sell', 1.0),
            1: ('sell', 0.75),
            2: ('sell', 0.5),
            3: ('sell', 0.25),
            4: ('hold', 0.0),
            5: ('buy', 0.25),
            6: ('buy', 0.5),
            7: ('buy', 0.75),
            8: ('buy', 1.0)
        }

        for i, (ticker, df) in enumerate(self.dfs.items()):
            action = actions[i]
            idx = self.data_indices[ticker]

            if idx >= len(df):
                done = True
                trade_info.append(f"Ticker {ticker} reached end of data. Ending episode.")
                break

            actual_price = df.iloc[idx]['Close_unscaled']

            action_type, proportion = action_mapping.get(action, ('hold', 0.0))

            if action_type == 'buy':
                adjusted_price = actual_price * (1 + self.slippage)
            elif action_type == 'sell':
                adjusted_price = actual_price * (1 - self.slippage)
            else:
                adjusted_price = actual_price

            fee = adjusted_price * self.transaction_fee

            buy_amount = 0
            sell_amount = 0

            reward = 0  
            if action_type == 'sell':
                if self.stock_owned[ticker]['quantity'] > 0:
                    sell_amount = int(self.stock_owned[ticker]['quantity'] * proportion)
                    sell_amount = max(1, sell_amount)  
                    sell_amount = min(sell_amount, self.stock_owned[ticker]['quantity'])  
                    proceeds = adjusted_price * sell_amount - fee * sell_amount
                    self.balance += proceeds
                    profit = (adjusted_price - self.stock_owned[ticker]['avg_price']) * sell_amount - fee * sell_amount
                    reward = profit  
                    self.stock_owned[ticker]['quantity'] -= sell_amount
                    if self.stock_owned[ticker]['quantity'] == 0:
                        self.stock_owned[ticker]['avg_price'] = 0
                    trade_info.append(f"Sell {sell_amount} of {ticker} at {adjusted_price:.2f}")
                else:
                    reward = invalid_action_penalty
                    trade_info.append(f"Cannot Sell {ticker} (No holdings)")
            elif action_type == 'buy':
                max_can_buy = min(
                    self.max_stock - self.stock_owned[ticker]['quantity'],
                    int(self.balance // (adjusted_price + fee))
                )
                buy_amount = int(max_can_buy * proportion)
                buy_amount = max(1, buy_amount)  
                buy_amount = min(buy_amount, self.max_stock - self.stock_owned[ticker]['quantity'], 
                                 int(self.balance // (adjusted_price + fee)))
                if buy_amount > 0:
                    cost = adjusted_price * buy_amount + fee * buy_amount
                    self.balance -= cost
                    total_quantity = self.stock_owned[ticker]['quantity'] + buy_amount
                    if total_quantity > 0:
                        self.stock_owned[ticker]['avg_price'] = (
                            (self.stock_owned[ticker]['avg_price'] * self.stock_owned[ticker]['quantity'] + adjusted_price * buy_amount)
                            / total_quantity
                        )
                    self.stock_owned[ticker]['quantity'] = total_quantity
                    reward = 0  
                    trade_info.append(f"Buy {buy_amount} of {ticker} at {adjusted_price:.2f}")
                else:
                    reward = invalid_action_penalty  
                    trade_info.append(f"Cannot Buy {ticker} (Insufficient balance or max stock)")
            else:
                trade_info.append(f"Hold {ticker}")

            total_reward += reward

            self.price_history[ticker].append(actual_price)

            self.data_indices[ticker] += 1

        self.trade_history.append(trade_info)

        self.portfolio_value = self.balance + sum(
            self.stock_owned[ticker]['quantity'] * self.stock_price[ticker] for ticker in self.dfs.keys()
        )
        self.total_asset.append(self.portfolio_value)

        if prev_portfolio_value > 0:
            portfolio_return = (self.portfolio_value - prev_portfolio_value) / prev_portfolio_value
        else:
            portfolio_return = 0

        if portfolio_return > 0.01:
            scaled_reward = portfolio_return * 150  
        else:
            scaled_reward = portfolio_return * 100

        total_reward += scaled_reward  

        if self.portfolio_value < 0:
            self.portfolio_value = 0
            self.balance = 0  
            done = True  

        if self.portfolio_value < self.initial_balance * (1 - self.max_loss):
            done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            done = True

        self.balance_history.append(self.balance)
        self.portfolio_value_history.append(self.portfolio_value)

        obs = self._next_observation()
        return obs, total_reward, done, {}

# PPO를 위한 액터-크리틱 신경망 정의
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim_list, seq_length=20):
        super(ActorCritic, self).__init__()
        self.seq_length = seq_length
        self.timesnet = TimesNet(input_dim=7, seq_length=seq_length).to(device)  
        self.policy_head = nn.ModuleList([nn.Linear(self.timesnet.embed_dim, action_dim) for action_dim in action_dim_list])
        self.value_head = nn.Linear(self.timesnet.embed_dim * len(action_dim_list), 1)  
        self.apply(self._weights_init)  

    def _weights_init(self, m):
        if isinstance(m, nn.Linear):
            nn.init.zeros_(m.bias)
            nn.init.xavier_uniform_(m.weight)

    def forward(self, x):
        stock_embeds = []
        for i in range(len(tickers)):
            start = i * (7 * self.seq_length + 1)
            end = start + 7 * self.seq_length
            seq = x[:, start:end]
            seq = seq.view(-1, self.seq_length, 7)
            embed = self.timesnet(seq)  
            stock_embeds.append(embed)
        policy_logits = [head(embed) for embed, head in zip(stock_embeds, self.policy_head)]  
        combined_embeds = torch.cat(stock_embeds, dim=1)  
        value = self.value_head(combined_embeds)  
        return policy_logits, value

    def act(self, state):
        state = state.to(device)
        policy_logits, _ = self.forward(state)
        actions = []
        action_logprobs = []
        for logits in policy_logits:
            dist = Categorical(logits=logits)
            action = dist.sample()
            actions.append(action.item())
            action_logprob = dist.log_prob(action)
            action_logprobs.append(action_logprob)
        return np.array(actions), torch.stack(action_logprobs)

    def evaluate(self, state, actions):
        policy_logits, value = self.forward(state)
        action_logprobs = []
        dist_entropies = []
        for i, logits in enumerate(policy_logits):
            dist = Categorical(logits=logits)
            action_logprob = dist.log_prob(actions[:, i])
            dist_entropy = dist.entropy()
            action_logprobs.append(action_logprob)
            dist_entropies.append(dist_entropy)
        return torch.stack(action_logprobs, dim=1), value.squeeze(-1), torch.stack(dist_entropies, dim=1)

# 메모리 클래스 정의
class Memory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear(self):
        self.states = []
        self.actions = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

# 진화 전략을 적용한 새로운 개체를 생성하는 클래스
class EvolutionStrategy:
    def __init__(self, population_size, mutation_rate=0.1, crossover_rate=0.5):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = []

    def initialize_population(self, policy_class, input_dim, action_dim_list, seq_length):
        self.population = [policy_class(input_dim, action_dim_list, seq_length) for _ in range(self.population_size)]

    def select_parents(self, fitness_scores):
        total_fitness = sum(fitness_scores)
        probabilities = [score / total_fitness for score in fitness_scores]
        parents = np.random.choice(self.population, size=2, p=probabilities, replace=False)
        return parents

    def crossover(self, parent1, parent2):
        child = ActorCritic(parent1.timesnet.embed_dim, [9] * env.stock_dim)
        for child_param, param1, param2 in zip(child.parameters(), parent1.parameters(), parent2.parameters()):
            prob = np.random.rand()
            if prob < self.crossover_rate:
                child_param.data.copy_(param1.data)
            else:
                child_param.data.copy_(param2.data)
        return child

    def mutate(self, policy):
        for param in policy.parameters():
            prob = np.random.rand()
            if prob < self.mutation_rate:
                param.data += torch.randn_like(param) * 0.02

    def evolve(self, fitness_scores):
        new_population = []
        for _ in range(self.population_size // 2):
            parent1, parent2 = self.select_parents(fitness_scores)
            child1 = self.crossover(parent1, parent2)
            child2 = self.crossover(parent2, parent1)
            self.mutate(child1)
            self.mutate(child2)
            new_population.extend([child1, child2])
        self.population = new_population

# PPO 트레이너 클래스 정의
class PPOTrainer:
    def __init__(self, env, policy, memory, optimizer, gamma=0.99, epsilon=0.2, epochs=10, entropy_coef=0.05):
        self.env = env
        self.policy = policy
        self.memory = memory
        self.optimizer = optimizer
        self.gamma = gamma
        self.epsilon = epsilon
        self.epochs = epochs
        self.entropy_coef = entropy_coef

    def train(self, max_episodes):
        for episode in range(max_episodes):
            state = self.env.reset()
            done = False
            total_reward = 0

            while not done:
                state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
                actions, action_logprobs = self.policy.act(state_tensor)
                next_state, reward, done, _ = self.env.step(actions)
                total_reward += reward
                self.memory.states.append(state)
                self.memory.actions.append(actions)
                self.memory.logprobs.append(action_logprobs.sum().item())
                self.memory.rewards.append(reward)
                self.memory.is_terminals.append(done)
                state = next_state

            self.ppo_update()
            self.memory.clear()

            self.plot_results(episode, total_reward)

    def ppo_update(self):
        if len(self.memory.states) == 0:
            return

        states = torch.tensor(self.memory.states, dtype=torch.float32).to(device)
        actions = torch.tensor(self.memory.actions, dtype=torch.int64).to(device)
        old_logprobs = torch.tensor(self.memory.logprobs, dtype=torch.float32).to(device)
        rewards = self.memory.rewards
        is_terminals = self.memory.is_terminals

        returns = self.compute_returns(rewards, is_terminals)

        advantages = self.compute_advantages(states, returns)

        for epoch in range(self.epochs):
            action_logprobs, state_values, dist_entropies = self.policy.evaluate(states, actions)
            total_logprobs = action_logprobs.sum(dim=1)
            total_entropies = dist_entropies.sum(dim=1)
            ratios = torch.exp(total_logprobs - old_logprobs)
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.epsilon, 1 + self.epsilon) * advantages
            value_loss = F.mse_loss(state_values.squeeze(-1), returns)
            loss = -torch.min(surr1, surr2) + 0.5 * value_loss - self.entropy_coef * total_entropies

            self.optimizer.zero_grad()
            loss_mean = loss.mean()
            loss_mean.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), max_norm=0.5)
            self.optimizer.step()

            print(f"Epoch {epoch+1}, Loss: {loss_mean.item():.4f}")

    def compute_returns(self, rewards, is_terminals):
        returns = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(rewards), reversed(is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            returns.insert(0, discounted_reward)
        return torch.tensor(returns, dtype=torch.float32).to(device)

    def compute_advantages(self, states, returns):
        with torch.no_grad():
            _, state_values = self.policy.forward(states)
            advantages = returns - state_values.squeeze(-1)
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        return advantages

    def plot_results(self, episode, total_reward):
        action_counts = np.zeros(9)
        for actions in self.env.action_history:
            action_counts += np.bincount(actions, minlength=9)

        plt.figure(figsize=(18, 12))
        plt.subplot(4, 1, 1)
        plt.plot(self.env.portfolio_value_history)
        plt.title(f'Episode {episode+1} - Portfolio Value Over Time')
        plt.ylabel('Portfolio Value')

        plt.subplot(4, 1, 2)
        action_labels = ['Sell 100%', 'Sell 75%', 'Sell 50%', 'Sell 25%', 'Hold', 
                        'Buy 25%', 'Buy 50%', 'Buy 75%', 'Buy 100%']
        plt.bar(action_labels, action_counts, color=['red', 'darkred', 'orange', 'lightcoral', 'gray', 
                                                    'lightgreen', 'green', 'darkgreen', 'lime'])
        plt.title('Action Distribution')
        plt.ylabel('Counts')
        plt.xticks(rotation=45)

        plt.subplot(4, 1, 3)
        initial_prices = {ticker: self.env.price_history[ticker][0] for ticker in tickers}
        final_prices = {ticker: self.env.price_history[ticker][-1] for ticker in tickers}
        returns = []
        for ticker in tickers:
            if initial_prices[ticker] == 0:
                ret = 0
            else:
                ret = (final_prices[ticker] - initial_prices[ticker]) / initial_prices[ticker] * 100
            returns.append(ret)
        plt.bar(tickers, returns, color=['blue', 'orange', 'purple'])
        plt.title('Stock Returns (%)')
        plt.ylabel('Return (%)')

        plt.subplot(4, 1, 4)
        hold_counts = [self.env.stock_owned[ticker]['quantity'] for ticker in tickers]
        plt.bar(tickers, hold_counts, color=['cyan', 'magenta', 'yellow'])
        plt.title('Current Holdings')
        plt.ylabel('Number of Stocks')

        plt.tight_layout()
        plt.show()

        print(f"Episode {episode+1} completed. Total Reward: {total_reward:.2f}, Final Portfolio Value: {self.env.portfolio_value_history[-1]:.2f}")
        print(f"Action counts:")
        for i, label in enumerate(action_labels):
            print(f"  {label}: {int(action_counts[i])}")
        print('-' * 50)

# 데이터 로드 및 환경 설정
tickers = ['AAPL', 'MSFT', 'GOOGL']
dfs = {}

for ticker in tickers:
    df = yf.download(ticker, start='2019-01-01', end='2023-12-31', progress=False)  
    df.loc[:, 'Close_unscaled'] = df['Close']  

    df.loc[:, 'MA10'] = df['Close'].rolling(window=10).mean()
    df.loc[:, 'MA20'] = df['Close'].rolling(window=20).mean()

    df.loc[:, 'Upper_Band'] = df['MA20'] + 2 * df['Close'].rolling(window=20).std()
    df.loc[:, 'Lower_Band'] = df['MA20'] - 2 * df['Close'].rolling(window=20).std()

    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up / ema_down
    df.loc[:, 'RSI'] = 100 - (100 / (1 + rs))

    df = df.dropna().copy()  

    scaler = StandardScaler()
    feature_cols = ['Close', 'MA10', 'MA20', 'RSI', 'Volume', 'Upper_Band', 'Lower_Band']
    df.loc[:, feature_cols] = scaler.fit_transform(df[feature_cols])

    dfs[ticker] = df.reset_index(drop=True)

env = MultiStockTradingEnv(dfs, initial_balance=5000000, seq_length=20)

# 진화 전략 설정
es = EvolutionStrategy(population_size=10)
es.initialize_population(ActorCritic, env.observation_space.shape[0], [9] * env.stock_dim, seq_length=env.seq_length)

# 진화 알고리즘과 PPO 통합 학습
for generation in range(50):
    fitness_scores = []
    for policy in es.population:
        optimizer = optim.Adam(policy.parameters(), lr=1e-5)  
        memory = Memory()
        trainer = PPOTrainer(env, policy, memory, optimizer)
        trainer.train(max_episodes=5)
        fitness_scores.append(env.portfolio_value_history[-1])

    es.evolve(fitness_scores)

    print(f"Generation {generation + 1} completed. Best Portfolio Value: {max(fitness_scores):.2f}")
